In [1]:
# ============================================================
# Synthetic Drone Telemetry Generator
# Creates:
#   - train_normal.csv (NORMAL ONLY)
#   - test_with_anomalies.csv (NORMAL + ANOMALIES)
# ============================================================

import numpy as np
import pandas as pd

np.random.seed(42)

# ============================================================
# CONFIG
# ============================================================

TRAIN_SAMPLES = 3000
TEST_SAMPLES = 2000
ANOMALY_START = 1600   # anomalies begin here (end region)

# ============================================================
# NORMAL FLIGHT GENERATOR
# ============================================================

def generate_normal_flight(n):
    t = np.arange(n)

    data = {
        "altitude": 10 + 0.01 * t + np.random.normal(0, 0.2, n),
        "velocity_z": np.random.normal(0, 0.05, n),
        "accel_z": np.random.normal(0, 0.1, n),
        "roll": np.random.normal(0, 0.5, n),
        "pitch": np.random.normal(0, 0.5, n),
        "yaw": np.random.normal(0, 1.0, n),
        "motor1_rpm": 5200 + np.random.normal(0, 50, n),
        "motor2_rpm": 5200 + np.random.normal(0, 50, n),
        "motor3_rpm": 5200 + np.random.normal(0, 50, n),
        "motor4_rpm": 5200 + np.random.normal(0, 50, n),
        "battery_voltage": 16.8 - 0.0005 * t + np.random.normal(0, 0.02, n),
        "current_draw": 8 + np.random.normal(0, 0.3, n),
    }

    return pd.DataFrame(data)

# ============================================================
# ANOMALY INJECTION
# ============================================================

def inject_anomalies(df, start_idx):
    df = df.copy()
    n = len(df)

    df["ground_truth"] = 0

    for i in range(start_idx, n):
        df.loc[i, "altitude"] -= np.random.uniform(3, 6)
        df.loc[i, "velocity_z"] -= np.random.uniform(0.5, 1.5)
        df.loc[i, "accel_z"] += np.random.uniform(2, 5)

        failed_motor = np.random.choice(
            ["motor1_rpm", "motor2_rpm", "motor3_rpm", "motor4_rpm"]
        )
        df.loc[i, failed_motor] -= np.random.uniform(1500, 2500)

        df.loc[i, "current_draw"] += np.random.uniform(5, 10)
        df.loc[i, "battery_voltage"] -= np.random.uniform(0.5, 1.0)

        df.loc[i, "ground_truth"] = 1

    return df

# ============================================================
# CREATE TRAIN (NORMAL ONLY)
# ============================================================

train_df = generate_normal_flight(TRAIN_SAMPLES)
train_df.to_csv("train_normal.csv", index=False)

# ============================================================
# CREATE TEST (NORMAL + ANOMALIES)
# ============================================================

test_df = generate_normal_flight(TEST_SAMPLES)
test_df = inject_anomalies(test_df, ANOMALY_START)
test_df.to_csv("test_with_anomalies.csv", index=False)

# ============================================================
# SUMMARY
# ============================================================

print("Train samples:", len(train_df))
print("Test samples:", len(test_df))
print("Injected anomalies:", test_df["ground_truth"].sum())
print("Anomaly start index:", ANOMALY_START)


Train samples: 3000
Test samples: 2000
Injected anomalies: 400
Anomaly start index: 1600


In [5]:
# ============================================================
# RADD++ Drone Anomaly Detection
# Train on NORMAL data, Detect on TEST data
# ============================================================

import numpy as np
import pandas as pd

from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import pairwise_distances
from sklearn.metrics import classification_report # Moved from the end

# ============================================================
# CONFIG (EDIT HERE)
# ============================================================

TRAIN_PATH = "train_normal.csv"   # NORMAL ONLY
TEST_PATH  = "test.csv"           # UNKNOWN

TARGET_MAX_ANOMALY_RATE = 0.01
PERCENTILE = 99.7
VOTE_THRESHOLD = 0.4

WINDOW_SIZE = 50
STEP_SIZE = 10
WINDOW_THRESHOLD = 0.4

RANDOM_STATE = 42

# ============================================================
# UTILS
# ============================================================

def perc_thresh(x):
    return np.percentile(x, PERCENTILE)

# ============================================================
# 1️⃣ LOAD + SCALE (TRAIN)
# ============================================================

train_df = pd.read_csv(TRAIN_PATH)
X_train_raw = train_df.select_dtypes(include=[np.number]).values # Renamed for clarity

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw) # Scaled training data

print("[TRAIN] samples:", len(X_train))
# ============================================================
# 2️⃣ AUTO-SEARCH ON TRAIN (NORMAL ONLY)
# ============================================================

# ---- KMEANS ----
# Default
k = 2 
for k_val in [2, 3, 4]:
    kmeans = KMeans(n_clusters=k_val, random_state=RANDOM_STATE, n_init='auto')
    kmeans.fit(X_train)
    km_dist_train = np.min(
        pairwise_distances(X_train, kmeans.cluster_centers_), axis=1
    )
    # Save current k
    k = k_val 
    if (km_dist_train > perc_thresh(km_dist_train)).mean() <= TARGET_MAX_ANOMALY_RATE:
        break

km_thresh = perc_thresh(km_dist_train)
print("[KMeans] k =", k)

# ---- DBSCAN ----
# Initialize with valid defaults (safe fallback)
eps = 0.8
ms = 10 
found_db = False

for eps_val in [0.8, 1.2, 1.6]:
    for ms_val in [10, 20, 30]:
        dbscan = DBSCAN(eps=eps_val, min_samples=ms_val)
        labels = dbscan.fit_predict(X_train)
        
        # Check condition
        if (labels == -1).mean() <= TARGET_MAX_ANOMALY_RATE:
            eps = eps_val
            ms = ms_val
            found_db = True
            break
    if found_db:
        break

print("[DBSCAN] eps =", eps, "min_samples =", ms)

# ---- OPTICS ----
# Initialize with valid defaults
ms_o = 10
xi = 0.03
found_opt = False

for ms_o_val in [10, 20, 30]:
    for xi_val in [0.03, 0.05, 0.1]:
        optics = OPTICS(min_samples=ms_o_val, xi=xi_val)
        labels = optics.fit_predict(X_train)
        
        # Check condition
        if (labels == -1).mean() <= TARGET_MAX_ANOMALY_RATE:
            ms_o = ms_o_val
            xi = xi_val
            found_opt = True
            break
    if found_opt:
        break

print("[OPTICS] min_samples =", ms_o, "xi =", xi)

# ---- LOF ----
# Default
k_lof = 20
for k_lof_val in [20, 30, 40]:
    lof = LocalOutlierFactor(n_neighbors=k_lof_val, novelty=True)
    lof.fit(X_train)
    
    # Save current k_lof
    k_lof = k_lof_val
    if (-lof.score_samples(X_train) > 0).mean() <= TARGET_MAX_ANOMALY_RATE:
        break

print("[LOF] neighbors =", k_lof)

# ---- OCSVM ----
# Initialize with valid defaults
nu = 0.005
gamma = 'scale'
found_svm = False

for nu_val in [0.005, 0.01, 0.02]:
    for gamma_val in ["scale", 0.1, 0.01]:
        ocsvm = OneClassSVM(nu=nu_val, gamma=gamma_val)
        ocsvm.fit(X_train)
        svm_train_score = -ocsvm.score_samples(X_train)
        
        # Check condition
        if (svm_train_score > perc_thresh(svm_train_score)).mean() <= TARGET_MAX_ANOMALY_RATE:
            nu = nu_val
            gamma = gamma_val
            found_svm = True
            break
    if found_svm:
        break

svm_thresh = perc_thresh(svm_train_score)
print("[OCSVM] nu =", nu, "gamma =", gamma)
# ============================================================
# 3️⃣ LOAD + SCALE (TEST)
# ============================================================

test_df = pd.read_csv(TEST_PATH)
FEATURE_COLS = train_df.select_dtypes(include=[np.number]).columns.tolist()

# The original code here re-read X_train_raw but didn't re-scale it.
# We must use the *fitted* scaler to transform the test data.

X_test_raw  = test_df[FEATURE_COLS].values

# FIX 2: Apply the previously fitted scaler to the test data.
# This is CRITICAL. The test data must be transformed using the same 
# scaling parameters (mean/median, range) learned from the training data.
X_test = scaler.transform(X_test_raw)


print("[TEST] samples:", len(X_test))

# ============================================================
# 4️⃣ SCORE TEST DATA
# ============================================================

# KMeans
km_dist_test = np.min(
    pairwise_distances(X_test, kmeans.cluster_centers_), axis=1
)
km_flag = (km_dist_test > km_thresh).astype(int)

# DBSCAN / OPTICS (re-fit on test for density)
# FIX 3: DBSCAN/OPTICS should use the parameters found on the training set,
# but they should be re-run on the test set to identify outliers (-1).
# The original code re-instantiated them and fit_predict() on X_test, which is generally 
# okay for density methods if the test set is large, but fitting them on the 
# training set and then using the `predict` or a similar method is safer
# in an anomaly detection pipeline.
# For simplicity and adherence to the spirit of the original code, we just
# ensure the parameters are correct and keep the fit_predict:

# DBSCAN is generally not used in a train/test pipeline this way; it should
# be re-run on the entire dataset or used with a clustering approach.
# In this specific context (RADD++) where density is part of the vote,
# re-running it on the test data alone to find 'sparse' points is acceptable
# for a simple ensemble, but keep the parameters found on the training set.
db_flag = (DBSCAN(eps=eps, min_samples=ms).fit_predict(X_test) == -1).astype(int)
opt_flag = (OPTICS(min_samples=ms_o, xi=xi).fit_predict(X_test) == -1).astype(int)

# LOF (uses the lof model fitted on X_train)
lof_flag = (lof.predict(X_test) == -1).astype(int)

# OCSVM (uses the ocsvm model fitted on X_train)
svm_test_score = -ocsvm.score_samples(X_test)
svm_flag = (svm_test_score > svm_thresh).astype(int)

# ============================================================
# 5️⃣ RADD VOTING (POINT LEVEL)
# ============================================================

flags = np.vstack([
    km_flag,
    db_flag,
    opt_flag,
    lof_flag,
    svm_flag
]).T

test_df["anomaly_score"] = flags.mean(axis=1)
test_df["anomaly"] = (test_df["anomaly_score"] >= VOTE_THRESHOLD).astype(int)

print("[POINT] anomalies:", test_df["anomaly"].sum())

# ============================================================
# 6️⃣ SLIDING WINDOW (TEST)
# ============================================================

scores = test_df["anomaly_score"].values
window_flags = np.zeros(len(scores))

for start in range(0, len(scores) - WINDOW_SIZE + 1, STEP_SIZE):
    end = start + WINDOW_SIZE
    if scores[start:end].mean() >= WINDOW_THRESHOLD:
        window_flags[start:end] = 1

test_df["window_anomaly"] = window_flags.astype(int)

print("[WINDOW] anomalies:", test_df["window_anomaly"].sum())

# Assumes 'ground_truth' column exists in test_df
print("\n--- Classification Report ---")
print(classification_report(
    test_df["ground_truth"],
    test_df["window_anomaly"]
))

[TRAIN] samples: 3000
[KMeans] k = 2
[DBSCAN] eps = 0.8 min_samples = 10
[OPTICS] min_samples = 10 xi = 0.03
[LOF] neighbors = 40
[OCSVM] nu = 0.005 gamma = scale
[TEST] samples: 2000
[POINT] anomalies: 472
[WINDOW] anomalies: 430

--- Classification Report ---
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1600
           1       0.93      1.00      0.96       400

    accuracy                           0.98      2000
   macro avg       0.97      0.99      0.98      2000
weighted avg       0.99      0.98      0.99      2000

